# Predicting Flight Delay

Problem Set-up:
We define a delayed flight to be one that is delayed by >= 15 minutes. 
The prediction problem is to train a model that can classify flights, to predict if they will or will not be delayed.

Use case:
- The idea is that this model would be useful to choosing airlines, flightpaths, airports, at the time of booking, relatively in advance of the scheduled departure (days, weeks, months ahead of time). Therefore, the prediction problem will focus on features that can be known in advance, rather than predicting using day-off features like weather and previous flights from that day. 

Notes:
- We restrict the analysis to relatively large airport, those with more than 20 (domestic) flights a day

# This notebook: more features
* one-hot day of week
* one-hot month
* one-hot airline
* one-hot airport
* one-hot departure hour

# Create separate models to predict for each airport 

Motivation:
The weights that should eb put on features (e.g. airlines) may differ depending on the airport (see e.g. NB 4B) since different airports can have different environments (e.g. San Diego and Chicago winters are very different; American Airlines is better in Tuscon than in Anchorage.

In [1]:
# Imports
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
sns.set_style('white')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

In [2]:
# Import custom code
from flightdelay.fld import io as flio

# Load data

In [3]:
airlines_df, airports_df, flights_df = flio.load_data()

In [4]:
# Filter data to keys of interest
keys = ['AIRLINE', 'FLIGHT_NUMBER','ORIGIN_AIRPORT','DEPARTURE_DELAY']
flights_df = flights_df[keys]

# Remove airports with less than a certain number of flights

In [5]:
all_airports, airport_inverse, airport_count = np.unique(flights_df['ORIGIN_AIRPORT'],return_counts=True,return_inverse=True)

In [6]:
# Determine number of flights for the origin airport
Nflights_orig = np.zeros(len(airport_inverse))
for i in range(len(all_airports)):
    Nflights_orig[np.where(airport_inverse==i)] = airport_count[i]

In [7]:
flights_df = flights_df.loc[flights_df.index[Nflights_orig>=7300]]

# Remove cancelled flights

In [8]:
flights_df = flights_df.dropna()

In [35]:
flights_df.head(100)

,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DEPARTURE_DELAY,flnum
0,AS,98,ANC,-11.0,ANCAS98
1,AA,2336,LAX,-8.0,LAXAA2336
2,US,840,SFO,-2.0,SFOUS840
3,AA,258,LAX,-5.0,LAXAA258
4,AS,135,SEA,-1.0,SEAAS135
5,DL,806,SFO,-5.0,SFODL806
6,NK,612,LAS,-6.0,LASNK612
7,US,2013,LAX,14.0,LAXUS2013
8,AA,1112,SFO,-11.0,SFOAA1112
9,DL,1173,LAS,3.0,LASDL1173


# Make unique identifier for each flight

In [19]:
flights_df['FLIGHT_NUMBER'] = flights_df['FLIGHT_NUMBER'].astype(str)
flnums = flights_df[['ORIGIN_AIRPORT','AIRLINE','FLIGHT_NUMBER']].fillna('').sum(axis=1)
delays = flights_df['DEPARTURE_DELAY'].values > 15

In [21]:
len(flnums)

5266602

In [20]:
len(delays)

5266602

In [22]:
unique_flnum, flnum_idx = np.unique(flnums,return_inverse=True)

In [23]:
N_uniqueflnum = len(unique_flnum)
uniquefl_idxs = [0]*N_uniqueflnum
uniquefl_delayp = [0]*N_uniqueflnum
for i in range(N_uniqueflnum):
    uniquefl_delayp[i] = np.mean(delays[flnum_idx==i])

In [ ]:
df_flnum = pd.DataFrame.from_dict(feat_dict)

# Make train and val set

In [36]:
N_flights = 1000000
N_train = int(N_flights*.7)

np.random.seed(0)
flight_take_idx = np.random.permutation(np.arange(N_flights))

flnums_train = flnums[flight_take_idx[:N_train]].values
flnums_val = flnums[flight_take_idx[N_train:]].values

In [26]:
y_train = delays[flight_take_idx[:N_train]]
y_val = delays[flight_take_idx[N_train:]]

# Make dict for prediction

In [27]:
from collections import defaultdict
flnum_pred_dict = defaultdict(bool,zip(unique_flnum,np.array(uniquefl_delayp)>.5))

In [28]:
flnum_pred_dict

defaultdict(bool,
            {'ABQAA1024': False,
             'ABQAA1101': False,
             'ABQAA1184': False,
             'ABQAA1220': False,
             'ABQAA1234': False,
             'ABQAA1248': False,
             'ABQAA1282': False,
             'ABQAA1324': False,
             'ABQAA1345': False,
             'ABQAA1377': False,
             'ABQAA1394': False,
             'ABQAA1424': False,
             'ABQAA1500': False,
             'ABQAA1573': False,
             'ABQAA1662': False,
             'ABQAA1686': False,
             'ABQAA1732': False,
             'ABQAA1877': False,
             'ABQAA1907': False,
             'ABQAA2010': False,
             'ABQAA2042': False,
             'ABQAA2202': False,
             'ABQAA2214': False,
             'ABQAA2215': False,
             'ABQAA2229': False,
             'ABQAA2244': False,
             'ABQAA2252': False,
             'ABQAA2255': False,
             'ABQAA2341': False,
             'ABQAA2511':

In [29]:
flnum_pred_dict['ABQAA1023']

False

# Predict delay in validation set

In [41]:
N_val = len(y_val)
val_pred = [0]*N_val
for i in range(N_val):
    val_pred[i] = flnum_pred_dict[flnums_val[i]]

0
100000
200000


In [48]:
print('accuracy:',1-sum(abs(val_pred-y_val))/N_val)

/Users/scott/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.
  if __name__ == '__main__':


accuracy: 0.783613333333


In [51]:
1-sum(y_val)/300000

0.79214333333333331